In [1]:
# Train neural network to predict significant wave height from SAR spectra.
# Train with heteroskedastic regression uncertainty estimates.
# Author: Peter Sadowski, Dec 2020
import os, sys
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # Needed to avoid cudnn bug.
import numpy as np
import h5py

import tensorflow as tf
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

sys.path = ['../'] + sys.path
from sarhs.generator import SARGenerator
from sarhs.heteroskedastic import Gaussian_NLL, Gaussian_MSE


In [2]:
def define_model():
    # Low-level features.
    inputs = Input(shape=(72, 60, 2))
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = GlobalMaxPooling2D()(x)
    x = Dense(256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    cnn = Model(inputs, x)

    # High-level features.
    inp = Input(shape=(32, ))  # 'hsSM', 'hsWW3v2', 'hsALT', 'altID', 'target' -> dropped
    x = Dense(units=256, activation='relu')(inp)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dropout(0.5)(x)
    ann = Model(inputs=inp, outputs=x)
    
    # Combine
    combinedInput = concatenate([cnn.output, ann.output])
    x = Dense(256, activation="relu")(combinedInput)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu", name='penultimate')(x)  
    x = Dropout(0.5)(x)
    x = Dense(2, activation="softplus", name='output')(x)
    model = Model(inputs=[cnn.input, ann.input], outputs=x)
    return model


In [4]:
# Train
file_model = '../models/heteroskedastic_2017.h5'
model = define_model()
model.compile(loss=Gaussian_NLL, optimizer=Adam(lr=0.0001), metrics=[Gaussian_MSE])

# Dataset
batch_size = 128
epochs = 123
#filename = '../../data/alt/sar_hs.h5'
filename = '/mnt/tmp/psadow/sar/sar_hs.h5'
train = SARGenerator(filename=filename, 
                     subgroups=['2015_2016', '2017'], 
                     batch_size=batch_size)
valid = SARGenerator(filename=filename, subgroups=['2018'], batch_size=batch_size)
# filename = '/mnt/tmp/psadow/sar/sar_hs.h5'
# epochs = 25
# train = SARGenerator(filename=filename, 
#                      subgroups=['2015_2016', '2017', '2018'], # Train on all data without early stopping.
#                      batch_size=batch_size)

# Callbacks
# This LR schedule is slower than in the paper.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=1) 
check = ModelCheckpoint(file_model, monitor='val_loss', verbose=0,
                        save_best_only=True, save_weights_only=False,
                        mode='auto', save_freq='epoch')
stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, 
                     mode='auto', baseline=None, restore_best_weights=False)
clbks = [reduce_lr, check, stop]

history = model.fit(train,
                    epochs=epochs,
                    validation_data=valid,
                    callbacks=clbks,
                    verbose=1)


Epoch 1/123
4449/4449 [==============================] - 108s 24ms/step - loss: 0.6521 - Gaussian_MSE: 0.3641 - val_loss: 0.4954 - val_Gaussian_MSE: 0.1896 - lr: 1.0000e-04
Epoch 2/123
4449/4449 [==============================] - 87s 20ms/step - loss: 0.2711 - Gaussian_MSE: 0.1766 - val_loss: 0.3721 - val_Gaussian_MSE: 0.1433 - lr: 1.0000e-04
Epoch 3/123
4449/4449 [==============================] - 88s 20ms/step - loss: 0.1545 - Gaussian_MSE: 0.1507 - val_loss: 0.3046 - val_Gaussian_MSE: 0.1316 - lr: 1.0000e-04
Epoch 4/123
4449/4449 [==============================] - 88s 20ms/step - loss: 0.0874 - Gaussian_MSE: 0.1377 - val_loss: 0.3066 - val_Gaussian_MSE: 0.1312 - lr: 1.0000e-04
Epoch 5/123
4449/4449 [==============================] - 88s 20ms/step - loss: 0.0320 - Gaussian_MSE: 0.1295 - val_loss: 0.2606 - val_Gaussian_MSE: 0.1278 - lr: 9.0000e-05
Epoch 6/123
4449/4449 [==============================] - 88s 20ms/step - loss: -0.0157 - Gaussian_MSE: 0.1221 - val_loss: 0.2725 - val_Gaus